## Семинар 13: "Обучение с подкреплением 1"

ФИО: Намит Максим Михайлович

###  FrozenLake


<img src="http://vignette2.wikia.nocookie.net/riseoftheguardians/images/4/4c/Jack's_little_sister_on_the_ice.jpg/revision/latest?cb=20141218030206" alt="a random image to attract attention" style="width: 400px;"/>



In [1]:
import gym
import numpy as np 

#create a single game instance
env = gym.make("FrozenLake-v0")

#start new game
env.reset();

In [2]:
# display the game state
env.render()


SFFF
FHFH
FFFH
HFFG


### legend

![img](https://cdn-images-1.medium.com/max/800/1*MCjDzR-wfMMkS0rPqXSmKw.png)

### Gym interface

The three main methods of an environment are
* __reset()__ - reset environment to initial state, _return first observation_
* __render()__ - show current environment state (a more colorful version :) )
* __step(a)__ - commit action __a__ and return (new observation, reward, is done, info)
 * _new observation_ - an observation right after commiting the action __a__
 * _reward_ - a number representing your reward for commiting action __a__
 * _is done_ - True if the MDP has just finished, False if still in progress
 * _info_ - some auxilary stuff about what just happened. Ignore it for now

In [3]:
print("initial observation code:", env.reset())
print('printing observation:')
env.render()
print("observations:", env.observation_space, 'n=', env.observation_space.n)
print("actions:", env.action_space, 'n=', env.action_space.n)

initial observation code: 0
printing observation:

SFFF
FHFH
FFFH
HFFG
observations: Discrete(16) n= 16
actions: Discrete(4) n= 4


In [4]:
print("taking action 2 (right)")
new_obs, reward, is_done, _ = env.step(2)
print("new observation code:", new_obs)
print("reward:", reward)
print("is game over?:", is_done)
print("printing new state:")
env.render()

taking action 2 (right)
new observation code: 4
reward: 0.0
is game over?: False
printing new state:
  (Right)
SFFF
FHFH
FFFH
HFFG


In [5]:
action_to_i = {
    'left':0,
    'down':1,
    'right':2,
    'up':3
}

### Попробуйте походить по замерзшему озеру, не упав в дырку. 
* Каждый шаг вы с вероятностью __0.5__ будете двигаться в выбранном направлении и с вероятностью __0.5__ в случайном.
* Если упадете, используйте __env.reset()__ чтобы перезапустить __env__

In [6]:
env.reset()

0

In [7]:
env.step(action_to_i['right'])
env.render()

  (Right)
SFFF
FHFH
FFFH
HFFG


In [8]:
env.step(action_to_i['left'])
env.render()

  (Left)
SFFF
FHFH
FFFH
HFFG


## Задание: 
Следуя шаблонам функций реализуйте алгоритм Policy iteration.
Протестируйте его с помощью функции __evaluate_policy__.
Попробуйте разные значение для gamma и сравните результаты.

In [9]:
def run_episode(env, policy, gamma = 1.0, render = False):
    obs = env.reset()
    total_reward = 0
    step_idx = 0
    while True:
        if render:
            env.render()
        obs, reward, done , _ = env.step(int(policy[obs]))
        total_reward += (gamma ** step_idx * reward)
        step_idx += 1
        if done:
            break
    return total_reward

In [10]:
def evaluate_policy(env, policy, gamma = 1.0,  n = 100):
    scores = [
            run_episode(env, policy, gamma = gamma, render = False)
            for _ in range(n)]
    return np.mean(scores)

In [11]:
from functools import reduce

In [12]:
# Используя фиксированную value_function и жадную стратегию получите policy

def eval_f(x, y, v, gamma):
    return x + y[0] * (y[2] + gamma * v[y[1]])

def extract_policy(v, gamma = 1.0):
    policy = np.zeros(env.env.nS)
    for s in range(env.env.nS):
        actions = np.zeros(env.env.nA)
        for i, _ in enumerate(actions):
            temp = env.env.P[s][i]
            actions[i] = reduce(lambda s, el_n: eval_f(s, el_n, v, gamma), \
                         temp, 0)
        policy[s] = np.argmax(actions)
    return policy

In [13]:
# Используя фиксированную policy с помощью сэмплирования получите оценку для value_function с точностью 1e-10

def compute_policy_v(env, policy, gamma=1.0, eps=1e-10):
    last_v = np.zeros(env.env.nS)
    v = np.zeros(env.env.nS)
    cond = 1.0
    while (cond > eps):
        last_v = np.copy(v)
        delta = -1
        for s in range(env.env.nS):
            temp = env.env.P[s][policy[s]]
            v[s] = reduce(lambda s, el_n: eval_f(s, el_n, last_v, gamma), \
                         temp, 0)
            if (abs(v[s] - last_v[s]) > delta):
                delta = abs(v[s] - last_v[s])
        cond = delta
    return v

In [14]:
def policy_iteration(env, gamma = 1.0):
    policy = np.random.choice(np.arange(env.env.nA), env.env.nS)
    max_iterations = 100000
    for i in range(max_iterations):
        old_policy_v = compute_policy_v(env, policy, gamma)
        new_policy = extract_policy(old_policy_v, gamma)
        if (np.all(policy == new_policy)):
            break

        policy = new_policy
    return policy

In [15]:
gamma = [1.0, 0.9, 0.8, 0.7, 0.6]
for el in gamma:
    env = gym.make("FrozenLake-v0")
    optimal_policy = policy_iteration(env, gamma=el)
    policy_score = evaluate_policy(env, optimal_policy, el, n=100)
    print('policy for gamma = ', el, ' equals ', policy_score)

policy for gamma =  1.0  equals  0.79
policy for gamma =  0.9  equals  0.0560898246583737
policy for gamma =  0.8  equals  0.01537614016149536
policy for gamma =  0.7  equals  0.004174525785603295
policy for gamma =  0.6  equals  0.00032299972045425043


In [17]:
gamma = 1.0
env = gym.make("FrozenLake-v0")
optimal_policy = policy_iteration(env, gamma=gamma)
policy_score = evaluate_policy(env, optimal_policy, gamma, n=100)
print('policy for gamma = ', gamma, ' equals ', policy_score)
run_episode(env, optimal_policy, gamma, render=True)

policy for gamma =  1.0  equals  0.76

SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG


1.0